In [12]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="quick-starts-ws-142196")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-142196
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-142196


In [14]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
comp_cluster = "cpu-project1"

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# First verified not exist
try:
    comp_cluster = ComputeTarget(workspace=ws, name=comp_cluster)
    print('Exist!!')
except comp_cluster:
     # Not exist
    comp_machine = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    # Create cluster 
    comp_cluster = ComputeTarget.create(ws, comp_cluster, comp_machine)
    # Wait for all ok
    cpu_cluster.wait_for_completion(show_output=True)
    print("Azure Machine Learning Compute cluster attached")

# max_nodes should be no greater than 4.


Exist!!


In [20]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import shutil
import os

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        "--C" :        choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        "--max_iter" : choice(50,100,200,300)
    }
)

# Specify a Policy
#Your Code Here
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
 ### YOUR CODE HERE ###
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)
est = SKLearn(
    source_directory= script_folder,
    compute_target=comp_cluster,
    entry_script="train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling = ps, 
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs = 15,
    max_concurrent_runs = 3,
    policy = policy,
    estimator = est
)

In [21]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [22]:
hyperdrive_run.wait_for_completion(show_output = True)

RunId: HD_d577677d-f560-4140-ada0-fa4ee1fc7517
Web View: https://ml.azure.com/experiments/quick-starts-ws-142196/runs/HD_d577677d-f560-4140-ada0-fa4ee1fc7517?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-142196/workspaces/quick-starts-ws-142196

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-07T19:49:40.329165][API][INFO]Experiment created<END>\n""<START>[2021-04-07T19:49:40.880497][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2021-04-07T19:49:41.087343][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-04-07T19:49:41.1012746Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END><START>[2021-04-07T19:50:11.5283555Z][SCHEDULER][INFO]Scheduling job, id='HD_d577677d-f560-4140-ada0-fa4ee1fc7517_1'<END><START>[2021-04-07T19:50:11.5476918Z][SCHEDULER][INFO]The execution 

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_file_names())
hyperdrive_model = best_run.register_model(model_name = 'BankMarketingModel', model_path = 'outputs/model.joblib')
print(best_run)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(url)

In [ ]:
from train import clean_data
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
dataset = x.join(y)
train_data, test_data =  train_test_split(dataset, random_state = 223)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    enable_early_stopping = True,
    primary_metric= "AUC_weighted",
    training_data= train_data,
    label_column_name="y",
    enable_onnx_compatible_models=True,
    n_cross_validations= 3)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

remote_run = exp.submit(automl_config, show_output = False)
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
from azureml.automl.runtime.onnx_convert import OnnxConverter
best_run_model, onnx_model= remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")

In [ ]:
#Delete compute_target
compute_target.delete()